# Task 1.1 - Dataset Exploration

### Load training data in a dataframe

In [ ]:
import gdown
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from utils import TimeSeriesDataset, collate_fn, load_data

In [ ]:
data_url = "https://drive.google.com/drive/folders/1MyX__3hRiPkWHGFKqZXpOyq32KQpPDTR?usp=sharing"
gdown.download_folder(data_url, quiet=False)

In [ ]:
ecg_data_path = 'data/X_train.bin'
labels_path = 'data/y_train.csv'

ecg_data = load_data(ecg_data_path, 'rb')
labels = load_data(labels_path, 'r')

In [ ]:
df = pd.DataFrame({'Data': ecg_data, 'Class': labels})
df['Lengths'] = df['Data'].apply(lambda seq: len(seq))
print(f'Number of total data points: {df["Data"].count()}')
df.head()

## Plot some samples

In [ ]:
# normalize data points
sequences = df["Data"].to_numpy()
concat = np.concatenate(sequences).reshape(-1, 1)
scaler = StandardScaler()
scaler.fit(concat)
normalized_data = [scaler.transform(seq.reshape(-1, 1)).flatten() for seq in sequences]


def plot_sequences(data, labels, num_rows, num_cols, start_idx=0, total_sequences=10, xlim_right=1200, title="Plots"):
  
  assert 0 <= start_idx < len(data)
  assert 0 < total_sequences and total_sequences == num_rows * num_cols
  # assert start_idx + total_sequences < len(data)

  fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5 + num_rows))
  sequences = data[start_idx : start_idx + total_sequences]
  classes = labels[start_idx : start_idx + total_sequences]
  sorted_idcs = np.argsort(classes)

  offset = 0
  for i in range(num_rows):
    for j in range(num_cols):
      seq = sequences[sorted_idcs[offset]]
      clss = classes[sorted_idcs[offset]]
      axs[i, j].plot(seq, label=clss)
      axs[i, j].set_xlim(0, xlim_right)
      axs[i, j].legend()
      offset += 1
  
  fig.suptitle(title, fontsize=16, y=0.95)
  plt.show()

In [ ]:
plot_sequences(
    normalized_data, 
    df["Class"].to_numpy(), 
    num_rows=2, num_cols=4, 
    start_idx=90, total_sequences=8, 
    xlim_right=800, title="ECG-Signals of different classes"
)

## Plot class distribution

In [ ]:
classes = ['Class 0', 'Class 1', 'Class 2', 'Class 3']

_df = df.groupby('Class').count()
count = _df['Data'].to_numpy()
print(f"Class 0: {count[0]}\tClass 1: {count[1]}\tClass 2: {count[2]}\tClass 3: {count[3]}")

fig, ax = plt.subplots()
ax.pie(count, labels=classes, autopct='%1.1f%%')
plt.title('Class Distribution of ECG Data')
plt.show()

## Analyze the lengths of the samples

- This information is relevant for model-selection and pre-processing of data for model


In [ ]:
unique_lengths, counts = np.unique(df['Lengths'], return_counts=True)
min_val = unique_lengths.min()
max_val = unique_lengths.max()
mean_val = unique_lengths.mean()
most_frequent_length = unique_lengths[np.argmax(counts)]

# print(np.max(counts), counts[np.argmax(counts)])

print(f'Number of all different lengths: {len(unique_lengths)}\n')
print("--- Absolute Lengths ---")
print(f'Min Length: {min_val}')
print(f'Max Length: {max_val}')
print(f'Avg. Length: {mean_val:.2f}')
print(f'Most Frequent Length: {most_frequent_length} with total of {np.max(counts)} occurences\n')

print("--- Class dependant Lengths ---")
mins = df.groupby("Class")["Lengths"].min()
maxs = df.groupby("Class")["Lengths"].max()
avgs = df.groupby("Class")["Lengths"].mean()
print(f"Class 0  --  Min-Length: {mins[0]}  -  Max-Length: {maxs[0]}  -  Avg-Length: {int(avgs[0])}")
print(f"Class 1  --  Min-Length: {mins[1]}  -  Max-Length: {maxs[1]}  -  Avg-Length: {int(avgs[1])}")
print(f"Class 2  --  Min-Length: {mins[2]}  -  Max-Length: {maxs[2]}  -  Avg-Length: {int(avgs[2])}")
print(f"Class 3  --  Min-Length: {mins[3]}  -  Max-Length: {maxs[3]}  -  Avg-Length: {int(avgs[3])}")

## Compute descriptive statistics

In [ ]:
def get_class_statistics(df: pd.DataFrame, class_: int):

    group = df.groupby("Class")["Data"].get_group(class_)
    all_stats = np.zeros(8)

    for series in group:
        stats = pd.Series(series).describe()
        all_stats += stats

    all_stats /= group.shape[0]
    return all_stats

# df.groupby("Class")["Data"].apply()
count0, mean0, std0, min0, quant0_25, quant0_50, quant0_75, max0 = get_class_statistics(df, 0)
count1, mean1, std1, min1, quant1_25, quant1_50, quant1_75, max1 = get_class_statistics(df, 1)
count2, mean2, std2, min2, quant2_25, quant2_50, quant2_75, max2 = get_class_statistics(df, 2)
count3, mean3, std3, min3, quant3_25, quant3_50, quant3_75, max3 = get_class_statistics(df, 3)

In [ ]:
print(f"-- Class 0 -- \n\nCount: {int(count0)}\nMean: {mean0:.2f}\nMin: {min0:.2f}\nMax: {max0:.2f}\n25% Quantile: {quant0_25:.2f}\n50% Quantile: {quant0_50:.2f}\n75% Quantile: {quant0_75:.2f}\n")
print(f"-- Class 1 -- \n\nCount: {int(count1)}\nMean: {mean1:.2f}\nMin: {min1:.2f}\nMax: {max1:.2f}\n25% Quantile: {quant1_25:.2f}\n50% Quantile: {quant1_50:.2f}\n75% Quantile: {quant1_75:.2f}\n")
print(f"-- Class 2 -- \n\nCount: {int(count2)}\nMean: {mean2:.2f}\nMin: {min2:.2f}\nMax: {max2:.2f}\n25% Quantile: {quant2_25:.2f}\n50% Quantile: {quant2_50:.2f}\n75% Quantile: {quant2_75:.2f}\n")
print(f"-- Class 3 -- \n\nCount: {int(count3)}\nMean: {mean3:.2f}\nMin: {min3:.2f}\nMax: {max3:.2f}\n25% Quantile: {quant3_25:.2f}\n50% Quantile: {quant3_50:.2f}\n75% Quantile: {quant3_75:.2f}")

In [ ]:
# plot mean distribution
max_lengths = 18286
df_norm = pd.DataFrame({"Data": normalized_data, "Class": df["Class"]})
df_norm["Data"] = df_norm["Data"].apply(lambda sample: np.pad(sample, (0, max_lengths - sample.shape[0])))
mean_signals = [np.mean(np.stack(df_norm[df_norm["Class"] == i]["Data"].to_numpy()), axis=0) for i in range(len(classes))]
plt.figure(figsize=(12, 6))

for mean_signal, clss in zip(mean_signals, classes):
    plt.plot(mean_signal, label=clss)

plt.title("Averaged signals per class")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.xlim(0, 600)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

### Create Training and Validation Split (80/20)

In [ ]:
# create training and validation split
X_train, X_validation, y_train, y_validation = train_test_split(ecg_data, labels, test_size=0.2, random_state=0, stratify=labels)

# X_train = [np.array(x, dtype=np.float32) for x in X_train]
# X_validation = [np.array(x, dtype=np.float32) for x in X_validation]
train_lengths = [x.shape[0] for x in X_train]
validation_lengths = [x.shape[0] for x in X_validation]


# verify that the training and validation split is proportional to the original class distribution
counter0 = Counter(labels)
counter1 = Counter(y_train)
counter2 = Counter(y_validation)

total0 = len(labels)
ratios0 = np.array([counter0[0]/total0, counter0[1]/total0, counter0[2]/total0, counter0[3]/total0]) # original dataset ratios

total1 = counter1[0] + counter1[1] + counter1[2] + counter1[3]
ratios1 = np.array([counter1[0]/total1, counter1[1]/total1, counter1[2]/total1, counter1[3]/total1]) # training dataset ratios

total2 = counter2[0] + counter2[1] + counter2[2] + counter2[3]
ratios2 = np.array([counter2[0]/total2, counter2[1]/total2, counter2[2]/total2, counter2[3]/total2]) # validation dataset ratios

assert np.allclose(ratios0, ratios1, atol=1e-3)
assert np.allclose(ratios0, ratios2, atol=1e-3)
assert np.allclose(ratios1, ratios2, atol=1e-3)


# prepare data for training
# X_train, y_train = torch.tensor(zero_pad_data(X_train, max_val), dtype=torch.float32), torch.tensor(y_train)
# X_validation, y_validation = torch.tensor(zero_pad_data(X_validation, max_val), dtype=torch.float32), torch.tensor(y_validation)

# Task 1.2 - Modeling and Tuning

Define the actual model architecture

In [ ]:
class ECGNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(ECGNN, self).__init__()

        # define convolutional layers (Conv2d) to extract local patterns in sequences
        # input --> stft --> Conv2d --> BatchNorm2d --> ReLu --> MaxPool2d
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.mp = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()

        # define LSTM to extract global patterns in sequences
        self.rnn = nn.LSTM(832, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x: torch.Tensor, lengths: torch.Tensor):
        
        n_fft, hop_length = 128, 64
        window = torch.hann_window(n_fft).to(x.device)
        x = torch.stft(x, n_fft=n_fft, hop_length=hop_length, window=window, return_complex=True).abs()

        x = x.unsqueeze(dim=1)
        x = self.mp(self.relu(self.bn1(self.conv1(x))))
        x = self.mp(self.relu(self.bn2(self.conv2(x)))) # output-shape: [64, 64, 62, 33] (batch_size, channels, freq_bins, time_bins)
        
        lengths = (lengths / hop_length).floor()
        lengths = lengths - 4
        lengths = (lengths / 4).floor()
        
        # x = torch.log2(x + 1e-6)
        
        x = x.view(x.size(0), -1, x.size(3))
        x = x.permute(0, 2, 1)
        lengths = lengths.clamp(max=x.size(1)).long()
        x = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)

        _, (hn, cn) = self.rnn(x)
        x = self.fc(hn[-1])
        
        return x


Define the training and test routine

In [ ]:
def train(
        model: nn.Module, 
        device: torch.device, 
        train_loader: torch.utils.data.DataLoader, 
        criterion: nn.CrossEntropyLoss,
        optimizer: optim.Optimizer, 
        epoch: int
):
    model.train()
    for batch_idx, (data, target, lengths) in enumerate(train_loader):
        # forward pass
        data, target, lengths = data.to(device), target.to(device), lengths.to(device)
        optimizer.zero_grad()
        output = model(data, lengths)
        # backward pass
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            data_processed = batch_idx * len(data)
            total_data = len(train_loader.dataset)
            progress = (100. * batch_idx) / len(train_loader)
            print("Train Epoch: {} [{}/{} ({:.0f}%)] \t Loss: {:.6f}".format(
                epoch, data_processed, total_data, progress, loss.item()
            ), end="\r")

def validate(
        model: nn.Module, 
        device: torch.device, 
        validation_loader: torch.utils.data.DataLoader,
        criterion: nn.CrossEntropyLoss
):
    model.eval()
    validation_loss = 0.
    correct = 0
    correct_per_class = torch.zeros(4)
    with torch.no_grad():
        for batch_idx, (data, target, lengths) in enumerate(validation_loader):
            # do forward pass
            data, target, lengths = data.to(device), target.to(device), lengths.to(device)
            output = model(data, lengths)
            validation_loss += criterion(output, target).item() * len(data)
            # compute correct predictions
            preds = output.argmax(dim=1, keepdim=False)
            mask = preds == target
            correct_preds = torch.count_nonzero(mask).item()
            correct += correct_preds
            for t, m in zip(target, mask):
                if m: correct_per_class[t] += 1

    total_data = len(validation_loader.dataset)
    validation_loss /= total_data
    progress = (100. * correct)  / len(validation_loader.dataset)
    
    print("Validation Set: Average Loss {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
        validation_loss, correct, total_data, progress
    ), end="\r")
    print("Class 0: {:.0f}/{}\tClass 1: {:.0f}/{}\tClass 3: {:.0f}/{}\tClass 3: {:.0f}/{}\n".format(
        correct_per_class[0], counter2[0], correct_per_class[1], counter2[1], correct_per_class[2], counter2[2], correct_per_class[3], counter2[3],
    ))
    
    return validation_loss

def evaluate_model(
        model: nn.Module,
        device: torch.device,
        test_loader: torch.utils.data.DataLoader,
        y_true: torch.Tensor,
        target_names: list
):
    model.eval()
    y_pred = torch.empty(0)
    
    with torch.no_grad():
        for data, _ in test_loader:
            # forward pass
            data = data.to(device)
            output = model(data)
            # compute correct predictions
            new_pred = output.argmax(dim=1, keepdim=False)
            y_pred = torch.cat((y_pred, new_pred))
    
    classification_report(y_true, y_pred, target_names=target_names)
    

In [ ]:
num_samples_per_class = torch.from_numpy(df.groupby("Class").count()["Data"].to_numpy())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# hyperparameters
weights =  num_samples_per_class.sum() / (num_samples_per_class * len(num_samples_per_class))
num_epochs = 20
batch_size = 32
lr = 0.001
weight_decay = 0.0001

input_size = 129
hidden_size = 64
num_layers = 1
output_size = 4

# initialize model and define necessary objects for training
model = ECGNN(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.CrossEntropyLoss(weight=weights.float())
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay) # try lr: 0.01 or 0.001 and other weight decay
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=4)

training_data = TimeSeriesDataset(X_train, y_train, train_lengths)
validation_data = TimeSeriesDataset(X_validation, y_validation, validation_lengths)

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
validation_loader = DataLoader(validation_data, batch_size=batch_size, collate_fn=collate_fn)

# training and test loop
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, criterion, optimizer, epoch)
    valid_loss = validate(model, device, validation_loader, criterion)
    scheduler.step(valid_loss)
    
evaluate_model(model, device, validation_loader, y_validation, classes)

In [221]:
'''
-- mit log2
Validation Set: Average Loss 1.2355, Accuracy: 660/1236 (53%)
Validation Set: Average Loss 1.2226, Accuracy: 280/1236 (23%)
Validation Set: Average Loss 1.1676, Accuracy: 222/1236 (18%)
Validation Set: Average Loss 1.2645, Accuracy: 150/1236 (12%)
Validation Set: Average Loss 1.2164, Accuracy: 255/1236 (21%)
Validation Set: Average Loss 1.1568, Accuracy: 392/1236 (32%)
Validation Set: Average Loss 1.1648, Accuracy: 411/1236 (33%)
Validation Set: Average Loss 1.1716, Accuracy: 292/1236 (24%)

-- ohne log2
Validation Set: Average Loss 1.1630, Accuracy: 626/1236 (51%)
Validation Set: Average Loss 1.1218, Accuracy: 411/1236 (33%)
Validation Set: Average Loss 1.1386, Accuracy: 362/1236 (29%)
Validation Set: Average Loss 1.0890, Accuracy: 675/1236 (55%)
Validation Set: Average Loss 1.1030, Accuracy: 527/1236 (43%)
Validation Set: Average Loss 1.1014, Accuracy: 662/1236 (54%)
Validation Set: Average Loss 1.0843, Accuracy: 528/1236 (43%)
Validation Set: Average Loss 1.0490, Accuracy: 697/1236 (56%)


-- mit min max scaler
Validation Set: Average Loss 1.3694, Accuracy: 695/1236 (56%)
Validation Set: Average Loss 1.3279, Accuracy: 699/1236 (57%)
Validation Set: Average Loss 1.2265, Accuracy: 583/1236 (47%)
Validation Set: Average Loss 1.1812, Accuracy: 241/1236 (19%)
Validation Set: Average Loss 1.1525, Accuracy: 258/1236 (21%)
Validation Set: Average Loss 1.1157, Accuracy: 339/1236 (27%)
Validation Set: Average Loss 1.0987, Accuracy: 408/1236 (33%)

-- ohne min max scaler
Validation Set: Average Loss 1.3544, Accuracy: 273/1236 (22%)
Validation Set: Average Loss 1.3171, Accuracy: 334/1236 (27%)
Validation Set: Average Loss 1.2616, Accuracy: 354/1236 (29%)
Validation Set: Average Loss 1.2030, Accuracy: 344/1236 (28%)
Validation Set: Average Loss 1.1772, Accuracy: 406/1236 (33%)
Validation Set: Average Loss 1.1630, Accuracy: 471/1236 (38%)
Validation Set: Average Loss 1.1514, Accuracy: 601/1236 (49%)
Validation Set: Average Loss 1.1464, Accuracy: 469/1236 (38%)
Validation Set: Average Loss 1.1340, Accuracy: 594/1236 (48%)
Validation Set: Average Loss 1.1242, Accuracy: 637/1236 (52%)
Validation Set: Average Loss 1.1233, Accuracy: 629/1236 (51%)
Validation Set: Average Loss 1.1218, Accuracy: 635/1236 (51%)
Validation Set: Average Loss 1.1187, Accuracy: 636/1236 (51%)
Validation Set: Average Loss 1.1180, Accuracy: 639/1236 (52%)
Validation Set: Average Loss 1.1161, Accuracy: 649/1236 (53%)
Validation Set: Average Loss 1.1168, Accuracy: 648/1236 (52%)
Validation Set: Average Loss 1.1173, Accuracy: 647/1236 (52%)
Validation Set: Average Loss 1.1156, Accuracy: 633/1236 (51%)
Validation Set: Average Loss 1.1143, Accuracy: 635/1236 (51%)
Validation Set: Average Loss 1.1137, Accuracy: 634/1236 (51%)


-------------------------------------------
num_epochs = 20
batch_size = 64
lr = 0.001
weight_decay = 0.0001
step_size = 10
gamma = 0.1

input_size = 257
hidden_size = 64
num_layers = 2
output_size = 4

Validation Set: Average Loss 1.0932, Accuracy: 640/1236 (52%)
Validation Set: Average Loss 1.0989, Accuracy: 624/1236 (50%)
Validation Set: Average Loss 1.0565, Accuracy: 671/1236 (54%)
Validation Set: Average Loss 1.0006, Accuracy: 595/1236 (48%)
Validation Set: Average Loss 0.9564, Accuracy: 638/1236 (52%)
Validation Set: Average Loss 0.9267, Accuracy: 671/1236 (54%)
Validation Set: Average Loss 1.0927, Accuracy: 783/1236 (63%)
Validation Set: Average Loss 0.9567, Accuracy: 707/1236 (57%)
Validation Set: Average Loss 0.9516, Accuracy: 733/1236 (59%)
Validation Set: Average Loss 1.0527, Accuracy: 774/1236 (63%)
Validation Set: Average Loss 0.9122, Accuracy: 789/1236 (64%)
Validation Set: Average Loss 0.9102, Accuracy: 770/1236 (62%)
Validation Set: Average Loss 0.9324, Accuracy: 778/1236 (63%)
Validation Set: Average Loss 0.9331, Accuracy: 779/1236 (63%)
Validation Set: Average Loss 0.9604, Accuracy: 816/1236 (66%)
Validation Set: Average Loss 0.9579, Accuracy: 813/1236 (66%)
Validation Set: Average Loss 0.9555, Accuracy: 803/1236 (65%)
Validation Set: Average Loss 0.9394, Accuracy: 786/1236 (64%)
Validation Set: Average Loss 0.9620, Accuracy: 800/1236 (65%)
Validation Set: Average Loss 0.9476, Accuracy: 797/1236 (64%)


-- bisher beste mit 78% accuracy

weights =  num_samples_per_class.max() / num_samples_per_class
num_epochs = 20
batch_size = 64
lr = 0.001
weight_decay = 0.0001
step_size = 10
gamma = 0.1

input_size = 129
hidden_size = 128
num_layers = 2
output_size = 4
'''

'\n-- mit log2\nValidation Set: Average Loss 1.2355, Accuracy: 660/1236 (53%)\nValidation Set: Average Loss 1.2226, Accuracy: 280/1236 (23%)\nValidation Set: Average Loss 1.1676, Accuracy: 222/1236 (18%)\nValidation Set: Average Loss 1.2645, Accuracy: 150/1236 (12%)\nValidation Set: Average Loss 1.2164, Accuracy: 255/1236 (21%)\nValidation Set: Average Loss 1.1568, Accuracy: 392/1236 (32%)\nValidation Set: Average Loss 1.1648, Accuracy: 411/1236 (33%)\nValidation Set: Average Loss 1.1716, Accuracy: 292/1236 (24%)\n\n-- ohne log2\nValidation Set: Average Loss 1.1630, Accuracy: 626/1236 (51%)\nValidation Set: Average Loss 1.1218, Accuracy: 411/1236 (33%)\nValidation Set: Average Loss 1.1386, Accuracy: 362/1236 (29%)\nValidation Set: Average Loss 1.0890, Accuracy: 675/1236 (55%)\nValidation Set: Average Loss 1.1030, Accuracy: 527/1236 (43%)\nValidation Set: Average Loss 1.1014, Accuracy: 662/1236 (54%)\nValidation Set: Average Loss 1.0843, Accuracy: 528/1236 (43%)\nValidation Set: Average